#### Question 1. Running Ollama with Docker. What's the version?

In [10]:
!ollama -v

ollama version is 0.1.48


#### Question 2. Downloading an LLM. Manifest file 

In [11]:
!cd /root/.ollama/models/manifests/registry.ollama.ai/library/gemma

/bin/bash: line 0: cd: /root/.ollama/models/manifests/registry.ollama.ai/library/gemma: Permission denied


In [12]:
!sudo chown $USER /root/.ollama/models/manifests/registry.ollama.ai/library/gemma

chown: cannot access '/root/.ollama/models/manifests/registry.ollama.ai/library/gemma': No such file or directory


In [13]:
!cd /root/.ollama/models/manifests/registry.ollama.ai/library

/bin/bash: line 0: cd: /root/.ollama/models/manifests/registry.ollama.ai/library: Permission denied


In [14]:
!sudo chown $USER /root/.ollama/models/manifests/registry.ollama.ai/library

chown: cannot access '/root/.ollama/models/manifests/registry.ollama.ai/library': No such file or directory


In [17]:
!cd /root/.ollama

/bin/bash: line 0: cd: /root/.ollama: Permission denied


In [ ]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [3]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=3
    )

    return results

In [5]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


In [6]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [8]:
llm('10 * 10')

'The code you provided is a simple Python program that calculates 10 multiplied by 10.\n\n```python\n10 * 10\n```\n\n**Output:**\n\n```\n100\n```\n\n**Explanation:**\n\n* The `10 * 10` expression performs the multiplication of 10 by 10, which is 100.\n* The `start_of_turn` and `end_of_turn` markers are used to indicate the start and end of a block of code.\n* The code is valid Python syntax and will execute the program as intended.'

In [9]:
print(_)

The code you provided is a simple Python program that calculates 10 multiplied by 10.

```python
10 * 10
```

**Output:**

```
100
```

**Explanation:**

* The `10 * 10` expression performs the multiplication of 10 by 10, which is 100.
* The `start_of_turn` and `end_of_turn` markers are used to indicate the start and end of a block of code.
* The code is valid Python syntax and will execute the program as intended.
